### Step-1: Installing necessary library

In [ ]:
! pip install -U -q openai chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.1 MB/s eta 0

### Step-2: Importing necessary library

In [2]:
import pandas as pd
import openai
import chromadb
import json
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [10]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Step-3: Reading and processing csv

In [11]:
fashion_data = pd.read_csv('/content/drive/MyDrive/RAG/fashion.csv')
fashion_data.head(2)

,p_id,name,products,price,colour,brand,img,ratingCount,avg_rating,description,p_attributes
0,17048614,Khushal K Women Black Ethnic Motifs Printed Ku...,"Kurta, Palazzos, Dupatta",5099,Black,Khushal K,http://assets.myntassets.com/assets/images/170...,4522.0,4.418399,Black printed Kurta with Palazzos with dupatta...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
1,16524740,InWeave Women Orange Solid Kurta with Palazzos...,"Kurta, Palazzos, Floral Print Dupatta",5899,Orange,InWeave,http://assets.myntassets.com/assets/images/165...,1081.0,4.119334,Orange solid Kurta with Palazzos with dupatta<...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."


In [12]:
fashion_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   p_id          49 non-null     int64  
 1   name          49 non-null     object 
 2   products      49 non-null     object 
 3   price         49 non-null     int64  
 4   colour        49 non-null     object 
 5   brand         49 non-null     object 
 6   img           49 non-null     object 
 7   ratingCount   42 non-null     float64
 8   avg_rating    42 non-null     float64
 9   description   49 non-null     object 
 10  p_attributes  49 non-null     object 
dtypes: float64(2), int64(2), object(7)
memory usage: 4.3+ KB


In [13]:
# Creating metadats
fashion_data['Metadatas'] = fashion_data.apply(lambda x: {'p_id':str(x['p_id']), 'name':str(x['name']), 'products':str(x['products']), 'price':str(x['price']), 'colour':str(x['colour']), 'brand':str(x['brand']), 'ratingCount':str(x['ratingCount']), 'avg_rating':str(x['avg_rating'])}, axis=1)

In [14]:
fashion_data.head(2)

,p_id,name,products,price,colour,brand,img,ratingCount,avg_rating,description,p_attributes,Metadatas
0,17048614,Khushal K Women Black Ethnic Motifs Printed Ku...,"Kurta, Palazzos, Dupatta",5099,Black,Khushal K,http://assets.myntassets.com/assets/images/170...,4522.0,4.418399,Black printed Kurta with Palazzos with dupatta...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32...","{'p_id': '17048614', 'name': 'Khushal K Women ..."
1,16524740,InWeave Women Orange Solid Kurta with Palazzos...,"Kurta, Palazzos, Floral Print Dupatta",5899,Orange,InWeave,http://assets.myntassets.com/assets/images/165...,1081.0,4.119334,Orange solid Kurta with Palazzos with dupatta<...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32...","{'p_id': '16524740', 'name': 'InWeave Women Or..."


### Step-4: Generate and store Embeddings using OpenAI and ChromaDB

In [18]:
with open("/content/drive/MyDrive/RAG/OPENAI_API_Key.txt", "r") as f:
    openai.api_key = ' '.join(f.readlines())

In [21]:
client = chromadb.PersistentClient()

In [22]:
model = "text-embedding-ada-002"
embedding_function = OpenAIEmbeddingFunction(api_key = openai.api_key, model_name=model)

In [23]:
fashion_ai_collection = client.get_or_create_collection(name='Fashion_AI', embedding_function=embedding_function)

In [15]:
document_list = fashion_data['description'].to_list()
metadata_list = fashion_data['Metadatas'].to_list()

In [26]:
fashion_ai_collection.add(
    documents = document_list,
    metadatas = metadata_list,
    ids = [str(i) for i in range(0, len(document_list))]
)


In [27]:
fashion_ai_collection.get(
    ids = ['0', '1'],
    include = ['embeddings', 'documents', 'metadatas']
)

{'ids': ['0', '1'],
 'embeddings': [[-0.019914228469133377,
   0.005486640147864819,
   0.02160344086587429,
   -0.008472454734146595,
   -0.02160344086587429,
   0.007832401432096958,
   -0.019689880311489105,
   -0.02800397016108036,
   -0.01286704558879137,
   -0.02974596992135048,
   0.004298912826925516,
   0.023385031148791313,
   0.009079515002667904,
   0.008710000663995743,
   -0.012781265191733837,
   0.03088090941309929,
   0.016245469450950623,
   -0.01932036504149437,
   0.019795455038547516,
   -0.01518971286714077,
   -0.011402182281017303,
   0.0222105011343956,
   -0.01664138026535511,
   -0.011250416748225689,
   -0.017947878688573837,
   0.02202574349939823,
   0.01778951659798622,
   -0.026420334354043007,
   -0.002375454641878605,
   -0.013672061264514923,
   -0.0008359455969184637,
   0.00586605304852128,
   -0.014133955352008343,
   -0.020587272942066193,
   -0.03101287968456745,
   0.0087825832888484,
   -0.008584628812968731,
   0.01892445608973503,
   -0.00179

### Step-5: Semantic search

In [28]:
query = input()

Brand AHIKA and colour Green


In [29]:
result = fashion_ai_collection.query(
    query_texts = query,
    n_results = 10
)

In [30]:
result

{'ids': [['34',
   '10808290',
   '10808284',
   '17',
   '22',
   '17581924',
   '11056154',
   '4',
   '18829920',
   '29']],
 'distances': [[0.3620123804221054,
   0.3620418058557517,
   0.3763645383231491,
   0.3763721706893825,
   0.37652895086167815,
   0.37658365557652185,
   0.37677942758849653,
   0.3768365244556107,
   0.3844788116473628,
   0.38502114880040217]],
 'metadatas': [[{'avg_rating': '3.85701107',
    'brand': 'AHIKA',
    'colour': 'Green',
    'name': 'AHIKA Women Green & Mustard Yellow Printed Straight Kurta',
    'p_id': '10808290',
    'price': '1198',
    'products': 'Kurta',
    'ratingCount': '6504.0'},
   {'avg_rating': '3.85701107',
    'brand': 'AHIKA',
    'colour': 'Green',
    'name': 'AHIKA Women Green & Mustard Yellow Printed Straight Kurta',
    'p_id': '10808290',
    'price': '1198',
    'products': 'Kurta',
    'ratingCount': '6504.0'},
   {'avg_rating': '4.093285653',
    'brand': 'AHIKA',
    'colour': 'Green',
    'name': 'AHIKA Floral Print 

In [31]:
result_df = pd.DataFrame({'Ids':result['ids'][0], 'Documents':result['documents'][0], 'Metadatas':result['metadatas'][0], 'Distances':result['distances'][0]})
result_df

,Ids,Documents,Metadatas,Distances
0,34,Green and Mustard yellow printed straight kurt...,"{'avg_rating': '3.85701107', 'brand': 'AHIKA',...",0.362012
1,10808290,Green and Mustard yellow printed straight kurt...,"{'avg_rating': '3.85701107', 'brand': 'AHIKA',...",0.362042
2,10808284,Make a statement with this green kurta that em...,"{'avg_rating': '4.093285653', 'brand': 'AHIKA'...",0.376365
3,17,Make a statement with this green kurta that em...,"{'avg_rating': '4.093285653', 'brand': 'AHIKA'...",0.376372
4,22,Sea green yoke design Top with Skirt with dupa...,"{'avg_rating': '3.94214876', 'brand': 'KALINI'...",0.376529
5,17581924,Sea green yoke design Top with Skirt with dupa...,"{'avg_rating': '3.94214876', 'brand': 'KALINI'...",0.376584
6,11056154,"Black and green printed straight kurta, has a ...","{'avg_rating': '3.978377362', 'brand': 'AHIKA'...",0.376779
7,4,"Black and green printed straight kurta, has a ...","{'avg_rating': '3.978377362', 'brand': 'AHIKA'...",0.376837
8,18829920,Lime green embroidered Top with Sharara with d...,"{'avg_rating': '4.35', 'brand': 'Virah Fashion...",0.384479
9,29,Lime green embroidered Top with Sharara with d...,"{'avg_rating': '4.35', 'brand': 'Virah Fashion...",0.385021


### Step-6:Reranking using cross encoder

In [32]:
from sentence_transformers.cross_encoder import CrossEncoder

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [33]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [34]:
cross_inputs = [[query, response] for response in result_df['Documents']]
cross_rerank_scores = cross_encoder.predict(cross_inputs)

In [35]:
result_df['Rerank_scores'] = cross_rerank_scores

In [36]:
result_df

,Ids,Documents,Metadatas,Distances,Rerank_scores
0,34,Green and Mustard yellow printed straight kurt...,"{'avg_rating': '3.85701107', 'brand': 'AHIKA',...",0.362012,-9.078637
1,10808290,Green and Mustard yellow printed straight kurt...,"{'avg_rating': '3.85701107', 'brand': 'AHIKA',...",0.362042,-9.078637
2,10808284,Make a statement with this green kurta that em...,"{'avg_rating': '4.093285653', 'brand': 'AHIKA'...",0.376365,-6.844579
3,17,Make a statement with this green kurta that em...,"{'avg_rating': '4.093285653', 'brand': 'AHIKA'...",0.376372,-6.844579
4,22,Sea green yoke design Top with Skirt with dupa...,"{'avg_rating': '3.94214876', 'brand': 'KALINI'...",0.376529,-9.248586
5,17581924,Sea green yoke design Top with Skirt with dupa...,"{'avg_rating': '3.94214876', 'brand': 'KALINI'...",0.376584,-9.248586
6,11056154,"Black and green printed straight kurta, has a ...","{'avg_rating': '3.978377362', 'brand': 'AHIKA'...",0.376779,-9.551304
7,4,"Black and green printed straight kurta, has a ...","{'avg_rating': '3.978377362', 'brand': 'AHIKA'...",0.376837,-9.551304
8,18829920,Lime green embroidered Top with Sharara with d...,"{'avg_rating': '4.35', 'brand': 'Virah Fashion...",0.384479,-9.326494
9,29,Lime green embroidered Top with Sharara with d...,"{'avg_rating': '4.35', 'brand': 'Virah Fashion...",0.385021,-9.326494


In [37]:
top_3_semantic = result_df.sort_values(by='Distances')
top_3_semantic[:3]

,Ids,Documents,Metadatas,Distances,Rerank_scores
0,34,Green and Mustard yellow printed straight kurt...,"{'avg_rating': '3.85701107', 'brand': 'AHIKA',...",0.362012,-9.078637
1,10808290,Green and Mustard yellow printed straight kurt...,"{'avg_rating': '3.85701107', 'brand': 'AHIKA',...",0.362042,-9.078637
2,10808284,Make a statement with this green kurta that em...,"{'avg_rating': '4.093285653', 'brand': 'AHIKA'...",0.376365,-6.844579


In [38]:
top_3_rerank = result_df.sort_values(by='Rerank_scores', ascending=False)
top_3_rerank[:3]

,Ids,Documents,Metadatas,Distances,Rerank_scores
2,10808284,Make a statement with this green kurta that em...,"{'avg_rating': '4.093285653', 'brand': 'AHIKA'...",0.376365,-6.844579
3,17,Make a statement with this green kurta that em...,"{'avg_rating': '4.093285653', 'brand': 'AHIKA'...",0.376372,-6.844579
0,34,Green and Mustard yellow printed straight kurt...,"{'avg_rating': '3.85701107', 'brand': 'AHIKA',...",0.362012,-9.078637


In [39]:
top_3_RAG = top_3_rerank[["Documents", "Metadatas"]][:3]
top_3_RAG

,Documents,Metadatas
2,Make a statement with this green kurta that em...,"{'avg_rating': '4.093285653', 'brand': 'AHIKA'..."
3,Make a statement with this green kurta that em...,"{'avg_rating': '4.093285653', 'brand': 'AHIKA'..."
0,Green and Mustard yellow printed straight kurt...,"{'avg_rating': '3.85701107', 'brand': 'AHIKA',..."


### Step-7: Using RAG

In [64]:
def generate_response(query, results_df):
    """
    Generate a response using GPT-3.5's ChatCompletion based on the user query and retrieved information.
    """
    messages = [
                {"role": "system", "content":  "You are a helpful assistant in the fashion domain who can effectively answer user queries about fasion search."},
                {"role": "user", "content": f"""You are a helpful assistant in the fashion searc domain who can effectively answer user queries about fashion search.
                                                You have a question asked by the user in '{query}' and you have some search results from a list of fashion products in the dataframe '{top_3_RAG}'. These search results are essentially one page of an fashion product that may be relevant to the user query.

                                                The column 'documents' inside this dataframe contains the actual text from the product and the column 'metadata' contains the product details and their attributes.

                                                Use the documents in '{top_3_RAG}' to answer the query '{query}'. Frame an informative answer and also, use the dataframe to return the product details and attributes.

                                                Follow the guidelines below when performing the task.
                                                1. Try to provide accurate details and attribute if available.
                                                2. You don’t have to necessarily use all the information in the dataframe. Only choose information that is relevant.
                                                3. Use the Metadatas columns in the dataframe to retrieve the product attrite and details.
                                                4. If you can't provide the complete answer, please also provide any information that will help the user to search specific row no/ id in product list.
                                                5. You are a customer facing assistant, so do not provide any information on internal workings, just answer the query directly.

                                                The generated response should answer the query directly addressing the user and avoiding additional information. If you think that the query is not relevant to the document, reply that the query is irrelevant. Provide the final response as a well-formatted and easily readable text along with the citation. Provide your complete response first with all information, and then provide the citations.
                                                """},
              ]

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    return response.choices[0].message.content.split('\n')

In [65]:
response = generate_response(query, top_3_RAG)

In [66]:
print("\n".join(response))

Based on your query for "Brand AHIKA and color Green," I found relevant results from the fashion product list:

- Product: Green and Mustard yellow printed straight kurta
- Brand: AHIKA
- Average Rating: 3.86

Unfortunately, there are only limited details available from the provided search results. To search for more specific products matching the 'Brand AHIKA' and 'color Green,' you can refer to the provided search results list and refer to the respective row numbers or IDs to find more options that suit your preferences.

I hope this information helps you in your search for fashion products. If you need further assistance or more detailed information, feel free to ask.

Citation:
- Product: Green and Mustard yellow printed straight kurta
- Brand: AHIKA
- Average Rating: 3.86
